<a href="https://colab.research.google.com/github/juliowatanabe/Alura_Google_AI/blob/main/Alura_Google_AI_Taroh.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Este programa ...

In [257]:
#Instalando o SDK do Google
!pip install -q -U google-generativeai

In [258]:
# Configurações iniciais
import google.generativeai as genai
import numpy as np
import pandas as pd
import random
# Importa o módulo userdata do google.colab para acessar o meu API_Key armazenado no Secrets
from google.colab import userdata

api_key = userdata.get("JKW_API_Key")

GOOGLE_API_KEY= api_key
genai.configure(api_key=GOOGLE_API_KEY)

In [259]:
# Listando os modelos disponíveis
for m in genai.list_models():
  if 'generateContent' in m.supported_generation_methods:
    print(m.name)

models/gemini-1.0-pro
models/gemini-1.0-pro-001
models/gemini-1.0-pro-latest
models/gemini-1.0-pro-vision-latest
models/gemini-1.5-pro-latest
models/gemini-pro
models/gemini-pro-vision


In [260]:
generation_config = {
  "candidate_count": 1,
  "temperature": 0.5,
}

In [261]:
safety_settings={
    'HATE': 'BLOCK_NONE',
    'HARASSMENT': 'BLOCK_NONE',
    'SEXUAL' : 'BLOCK_NONE',
    'DANGEROUS' : 'BLOCK_NONE'
    }

In [262]:
model = genai.GenerativeModel(model_name='gemini-1.5-pro-latest',
                                  generation_config=generation_config,
                                  safety_settings=safety_settings,)

In [263]:
carta = random.randint(1, 21)
response = model.generate_content(f"qual é a carta {carta} do Taroh.")
#response.text

In [264]:
significado = model.generate_content(f'Consulte o significado da Carta do Taroh {response}')

In [265]:
#significado.text

In [266]:
for m in genai.list_models():
  if 'embedContent' in m.supported_generation_methods:
    print(m.name)

models/embedding-001
models/text-embedding-004


In [267]:
title = "Taroh"
sample_text = significado.text

embeddings = genai.embed_content(model="models/embedding-001",
                                 content=sample_text,
                                 title=title,
                                 task_type="RETRIEVAL_DOCUMENT")

#print(embeddings)

In [268]:
DOCUMENT1 = {
    "Título": "Taroh",
    "Conteúdo": significado.text}

documents = [DOCUMENT1]

In [269]:
df = pd.DataFrame(documents)
df.columns = ["Titulo", "Conteudo"]
df

,Titulo,Conteudo
0,Taroh,A resposta fornecida pelo modelo está correta....


In [270]:
model = "models/embedding-001"

In [271]:
def embed_fn(title, text):
  return genai.embed_content(model=model,
                                 content=text,
                                 title=title,
                                 task_type="RETRIEVAL_DOCUMENT")["embedding"]

In [272]:
df["Embeddings"] = df.apply(lambda row: embed_fn(row["Titulo"], row["Conteudo"]), axis=1)
#df

In [273]:
def gerar_e_buscar_consulta(consulta, base, model):
  embedding_da_consulta = genai.embed_content(model=model,
                                 content=consulta,
                                 task_type="RETRIEVAL_QUERY")["embedding"]

  produtos_escalares = np.dot(np.stack(df["Embeddings"]), embedding_da_consulta)

  indice = np.argmax(produtos_escalares)
  return df.iloc[indice]["Conteudo"]

In [274]:
consulta = "qual é o significado da carta"

trecho = gerar_e_buscar_consulta(consulta, df, model)
#print(trecho)

In [275]:
generation_config = {
  "temperature": 0,
  "candidate_count": 1
}

In [276]:
prompt = f"Resuma esse texto de uma forma mais descontraída, sem adicionar informações que não façam parte do texto: {trecho}"

model_2 = genai.GenerativeModel("gemini-1.0-pro",
                                generation_config=generation_config)
response = model_2.generate_content(prompt)
print(response.text)

**O Enforcado: O Cara de Cabeça Pra Baixo**

Essa carta do Tarô é tipo o cara que tá pendurado de cabeça pra baixo, né? E isso significa que é hora de:

* **Ver o Mundo de Outro Jeito:** Tipo, dar uma virada na sua perspectiva e ver as coisas de um ângulo diferente.
* **Sacrificar Alguma Coisa:** Às vezes, você precisa abrir mão de algo pra ganhar algo melhor.
* **Relaxar e Aceitar:** Não adianta lutar contra a maré. Aceite a situação e confie que ela vai te levar aonde você precisa ir.
* **Conectar com Sua Intuição:** Essa carta te ajuda a ouvir sua voz interior e a encontrar a sabedoria que você precisa.

Claro, o significado pode mudar dependendo das outras cartas que aparecem na leitura e da sua intuição. Mas no geral, o Enforcado é um lembrete de que às vezes é preciso dar um passo atrás para dar dois passos à frente.
